In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
from scipy.stats import zscore, false_discovery_control
from pingouin import ancova
from sklearn.impute import SimpleImputer
from itertools import combinations
import sys

sys.path.append("../../")
from lib.general import get_stage_list
from lib.stats import demographic_characteristics
from lib.r_interface import tukey_multiple_dvs

Error importing in API mode: ImportError('On Windows, cffi mode "ANY" is only "ABI".')
Trying to import in ABI mode.
Trying to import in ABI mode.
c:\Users\m\Documents\medical-data\.venv\Lib\site-packages\rpy2\rinterface\__init__.py:1211: UserWarning: Environment variable "PATH" redefined by R and overriding existing variable. Current: "c:\Users\m\Documents\medical-data\.venv\Scripts;C:\Users\m\Documents\medical-data\.venv\Scripts;C:\VulkanSDK\1.4.328.1\Bin;C:\Python314\Scripts\;C:\Python314\;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v13.0\bin\x64;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v13.0\bin;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0\;C:\WINDOWS\System32\OpenSSH\;C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common;C:\Program Files (x86)\Windows Kits\10\Windows Performance Toolkit\;C:\Program Files\NVIDIA Corporation\Nsight Compute 2025.3.0\;C:\Program Files\CMake\bin;C:\Program Files\Git\cmd;C:\Program 

### Input

In [2]:
# Define I/O paths
path_input_demographics: Path = Path(
    "../../../data/processed/adni/demographics_biomarkers.csv"
).resolve()
path_input_dict: Path = Path("../../../data/processed/adni/somascan_dict.csv").resolve()
path_input_proteomics: Path = Path(
    "../../../data/processed/adni/somascan.csv"
).resolve()

In [3]:
# Read files
df: pd.DataFrame = pd.read_csv(path_input_demographics)
df_dict: pd.DataFrame = pd.read_csv(path_input_dict).convert_dtypes()
df_proteomics: pd.DataFrame = pd.read_csv(path_input_proteomics).convert_dtypes()

In [4]:
# Get stage list
stage_list: list[str] = get_stage_list(3)

In [5]:
# Define biomarker positivivity
df["ab_positive"] = (df["ab42"] < 976.6).astype(bool)
df["pet_positive"] = (df["av45"] > 1.11).astype(bool)
df["ptau_positive"] = (df["ptau"] > (df["ptau"].quantile(2 / 3))).astype(bool)
df["ttau_positive"] = (df["ttau"] > (df["ttau"].quantile(2 / 3))).astype(bool)

In [6]:
# 3-bit stage definition
df.loc[~df["ab_positive"] & ~df["pet_positive"] & ~df["ptau_positive"], "stage"] = (
    stage_list[0]
)
df.loc[df["ab_positive"] & ~df["pet_positive"] & ~df["ptau_positive"], "stage"] = (
    stage_list[1]
)
df.loc[df["ab_positive"] & df["pet_positive"] & ~df["ptau_positive"], "stage"] = (
    stage_list[3]
)
df.loc[df["ab_positive"] & df["pet_positive"] & df["ptau_positive"], "stage"] = (
    stage_list[5]
)
df.loc[~df["ab_positive"] & ~df["pet_positive"] & df["ptau_positive"], "stage"] = (
    stage_list[6]
)

df.loc[~df["ab_positive"] & df["pet_positive"] & ~df["ptau_positive"], "stage"] = (
    stage_list[2]
)
df.loc[~df["ab_positive"] & df["pet_positive"] & df["ptau_positive"], "stage"] = (
    stage_list[4]
)

df.loc[df["ab_positive"] & ~df["pet_positive"] & df["ptau_positive"], "stage"] = (
    "excluded"
)

In [7]:
# Convert stage to categorical
df["stage"] = pd.Categorical(df["stage"], categories=stage_list, ordered=True)
df: pd.DataFrame = (
    df.dropna().sort_values(by="RID", ascending=True).reset_index(drop=True)
)

In [8]:
# Join dataframes
df: pd.DataFrame = df.join(
    df_proteomics.set_index("RID"), on="RID", how="inner"
).reset_index(drop=True)

### Processing

In [9]:
# Get demographics characteristics
df_stats: pd.DataFrame = demographic_characteristics(df, stage_list)

In [10]:
df_stats

,Total,CSF-/PET-/pTau-,CSF+/PET-/pTau-,CSF-/PET+/pTau-,CSF+/PET+/pTau-,CSF-/PET+/pTau+,CSF+/PET+/pTau+,CSF-/PET-/pTau+,p_value
,,,,,,,,,
N,387,143,33,11,61,23,98,18,
Age (year),72.1 (7.2),70.9 (6.9),69.5 (8.0),70.0 (5.5),73.0 (6.5),75.4 (7.3),73.5 (6.9),73.6 (9.2),0.0030
BMI (kg/m^2),27.5 (4.7),28.2 (4.9),28.5 (5.3),29.5 (6.3),27.0 (4.3),25.9 (3.5),26.4 (4.5),27.9 (3.1),0.0203
Sex,,,,,,,,,
Female (n),173 (45%),68 (48%),10 (30%),8 (73%),20 (33%),12 (52%),44 (45%),11 (61%),0.0477
Male (n),214 (55%),75 (52%),23 (70%),3 (27%),41 (67%),11 (48%),54 (55%),7 (39%),
Cognitive Status,,,,,,,,,
CN (n),102 (26%),55 (38%),8 (24%),2 (18%),15 (25%),5 (22%),9 (9%),8 (44%),< 0.001
MCI (n),285 (74%),88 (62%),25 (76%),9 (82%),46 (75%),18 (78%),89 (91%),10 (56%),


In [11]:
stage_list = stage_list[:4]

In [12]:
df = df[df["stage"].isin(stage_list)]

In [13]:
# Convert bool columns to int
df[df.select_dtypes(include=[bool]).columns] = df.select_dtypes(include=[bool]).astype(
    int
)

In [14]:
# For each protein, perform ANCOVA, and store the proteins that differ significantly between stages
protein_differ_between_stages: list[str] = []
for protein in df_dict["label"].unique():
    df_ancova: pd.DataFrame = ancova(
        data=df,
        dv=protein,
        between="stage",
        covar=["age", "bmi", "sex", "cog", "apoe4"],
        effsize="np2",
    )
    if df_ancova["p-unc"].values[0] < 0.05:
        protein_differ_between_stages.append(protein)

c:\Users\m\Documents\medical-data\.venv\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 6, but rank is 3
  warnings.warn('covariance of constraints does not have full '
c:\Users\m\Documents\medical-data\.venv\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 6, but rank is 3
  warnings.warn('covariance of constraints does not have full '
c:\Users\m\Documents\medical-data\.venv\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 6, but rank is 3
  warnings.warn('covariance of constraints does not have full '
c:\Users\m\Documents\medical-data\.venv\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 6, but rank is 3
  wa

### Tukey HSD post hoc

In [15]:
# Z-transform the data
data_standardized: pd.DataFrame | np.ndarray = zscore(
    df[df_dict["label"]].astype(float), axis=0, nan_policy="omit"
)
# Remove outliers using z-score threshold of 3
data_standardized[data_standardized > 3] = np.nan

In [16]:
# Impute missing values using mean
imputer: SimpleImputer = SimpleImputer(strategy="mean")
data_imputed: np.ndarray = imputer.fit_transform(data_standardized)
df[df_dict["label"]] = data_imputed

In [17]:
# Perform Tukey's HSD test for proteins that differ significantly between stages
# Process 200 proteins at a time, and append the results to a dataframe
df_result: pd.DataFrame = pd.DataFrame(columns=list(combinations(stage_list, 2)))
for i in range(0, len(protein_differ_between_stages), 200):
    proteins: list[str] = protein_differ_between_stages[i : i + 200]
    df_tukey: pd.DataFrame = tukey_multiple_dvs(df, proteins, stage_list)
    df_result: pd.DataFrame = pd.concat([df_result, df_tukey.dropna(how="all")], axis=0)

R callback write-console: Loading required package: MASS
  
R callback write-console: 
Attaching package: 'TH.data'

  
R callback write-console: The following object is masked from 'package:MASS':

    geyser

  
R callback write-console: 
Attaching package: 'TH.data'

  
R callback write-console: The following object is masked from 'package:MASS':

    geyser

  
C:\Users\m\AppData\Local\Temp\ipykernel_12180\3952000429.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_result: pd.DataFrame = pd.concat([df_result, df_tukey.dropna(how="all")], axis=0)
C:\Users\m\AppData\Local\Temp\ipykernel_12180\3952000429.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this 

In [18]:
# Display the number of proteins that differ significantly between each pair of stages
df_result[df_result < 0.05].dropna(axis=0, how="all").notna().sum(axis=0)

(CSF-/PET-/pTau-, CSF+/PET-/pTau-)    1792
(CSF-/PET-/pTau-, CSF-/PET+/pTau-)      56
(CSF-/PET-/pTau-, CSF+/PET+/pTau-)    1460
(CSF+/PET-/pTau-, CSF-/PET+/pTau-)    1193
(CSF+/PET-/pTau-, CSF+/PET+/pTau-)      34
(CSF-/PET+/pTau-, CSF+/PET+/pTau-)     909
dtype: int64

### FDR correction

In [19]:
# For every column, perform BH correction
df_result_adj: pd.DataFrame = df_result.apply(false_discovery_control, axis=0)

In [20]:
df_result_adj[df_result_adj < 0.05].dropna(axis=0, how="all").notna().sum(axis=0)

(CSF-/PET-/pTau-, CSF+/PET-/pTau-)    1653
(CSF-/PET-/pTau-, CSF-/PET+/pTau-)       0
(CSF-/PET-/pTau-, CSF+/PET+/pTau-)    1171
(CSF+/PET-/pTau-, CSF-/PET+/pTau-)     788
(CSF+/PET-/pTau-, CSF+/PET+/pTau-)       0
(CSF-/PET+/pTau-, CSF+/PET+/pTau-)     486
dtype: int64

In [21]:
pairwise_comparisons: list[tuple[str, str]] = list(combinations(stage_list, 2))

In [22]:
df_mat: pd.DataFrame = df_result_adj[df_result_adj < 0.05]

In [23]:
intersection_count = np.zeros(
    [len(pairwise_comparisons), len(pairwise_comparisons)]
).astype(int)
for i in range(len(pairwise_comparisons)):
    for j in range(i + 1):
        intersection_count[i, j] = len(
            set(df_mat[[pairwise_comparisons[i]]].dropna().index).intersection(
                set(df_mat[[pairwise_comparisons[j]]].dropna().index)
            )
        )

In [24]:
intersection_count

array([[1653,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0],
       [ 988,    0, 1171,    0,    0,    0],
       [ 712,    0,  563,  788,    0,    0],
       [   0,    0,    0,    0,    0,    0],
       [ 430,    0,  427,  440,    0,  486]])